In [ ]:
from train_models_debug import single, sect1, sect2

In [ ]:
conv_layers = [64,128,256,32]
dense_layers = [16,256,128]
target = 4

In [ ]:
def reduce_model_size(conv_layers, dense_layers):
    # Reduce convolutional filters by 20%
    conv_layers = [max(1, int(layer * 0.8)) for layer in conv_layers if layer > 1]

    # Reduce dense units by 20%
    dense_layers = [max(1, int(layer * 0.8)) for layer in dense_layers if layer > 1]

    # Remove layers with size 1
    conv_layers = [layer for layer in conv_layers if layer >= 4]
    dense_layers = [layer for layer in dense_layers if layer >= 8]
    return conv_layers, dense_layers

In [ ]:
import matplotlib.pyplot as plt

# Lists to store trainable parameters and training times
trainable_params_list = []
training_time_list = []
iteration_list = []

for i in range(10):
    print(f"Training model with: {conv_layers}, {dense_layers}")

    net3 = sect1()
    trainable_params = net3.initialise_data_and_model(conv_layers=conv_layers, dense_layers=dense_layers)
    params = {'epochs': 20,
              'tensorboard': False, 
              'cp_callback': False,
              'weights': None,
              'stop_at': target,
             }
    reached_target, training_time = net3.train(params)

    print(f"Training time: {training_time}")
    trainable_params_list.append(trainable_params)
    training_time_list.append(training_time)
    iteration_list.append(i)

    # If the target is not reached, break the loop
    if reached_target == False:
        break

    # Reduce the model size for the next iteration
    conv_layers, dense_layers = reduce_model_size(conv_layers, dense_layers)

# Plot the first graph: Trainable parameters vs. Training time
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)  # Plot in the first subplot (1 row, 2 columns)
plt.plot(trainable_params_list, training_time_list, marker='o', linestyle='-', color='b')
plt.xlabel('Trainable Parameters')
plt.ylabel('Training Time (seconds)')
plt.title('Trainable Parameters vs Training Time')

# Plot the second graph: Iteration vs. Training time
plt.subplot(1, 2, 2)  # Plot in the second subplot
plt.plot(iteration_list, training_time_list, marker='x', linestyle='-', color='r')
plt.xlabel('Iteration')
plt.ylabel('Training Time (seconds)')
plt.title('Iteration vs Training Time')

# Show both plots
plt.tight_layout()
plt.show()